In [ ]:
import pandas as pd
import pickle
from pathlib import Path

from lib.util_plot import *
from lib.constants import *

import lib.H01_preprocessing.h01_constants as H01
from lib.pandas_impl import *
from lib.pandas_stats_impl import *
from lib.pandas_stats_H01 import H01AggregateStatistics
from lib.multilevel_analysis import MultilevelAnalysis
from models import *

### Prepare data

In [ ]:
data_folder = Path.cwd() / 'data' / 'H01'
eval_folder = Path.cwd() / 'data' / 'eval' / 'H01_24-12-17'
eval_folder.mkdir(parents=True, exist_ok=True)
plot_folder = eval_folder / "plots"
plot_folder.mkdir(parents=True, exist_ok=True)
synapse_file = data_folder / 'synapses_grid-25000_L3-region_aggregated.csv'

In [ ]:
df_synapses = pd.read_csv(synapse_file)

if("pre_celltype_merged" not in df_synapses.columns):
    df_synapses.loc[:, "pre_celltype_merged"] = H01.UNK_EXC_INH_OTHER[0]
    df_synapses.loc[df_synapses["pre_celltype"].isin(H01.EXC), "pre_celltype_merged"] = H01.UNK_EXC_INH_OTHER[1]
    df_synapses.loc[df_synapses["pre_celltype"].isin(H01.INH), "pre_celltype_merged"] = H01.UNK_EXC_INH_OTHER[2]
    df_synapses.loc[df_synapses["pre_celltype"].isin(H01.OTHER), "pre_celltype_merged"] = H01.UNK_EXC_INH_OTHER[3]

if("post_celltype_merged" not in df_synapses.columns):
    df_synapses.loc[:, "post_celltype_merged"] = H01.UNK_EXC_INH_OTHER[0]
    df_synapses.loc[df_synapses["post_celltype"].isin(H01.EXC), "post_celltype_merged"] = H01.UNK_EXC_INH_OTHER[1]
    df_synapses.loc[df_synapses["post_celltype"].isin(H01.INH), "post_celltype_merged"] = H01.UNK_EXC_INH_OTHER[2]
    df_synapses.loc[df_synapses["post_celltype"].isin(H01.OTHER), "post_celltype_merged"] = H01.UNK_EXC_INH_OTHER[3]

df_synapses_indexed = df_synapses.set_index(["pre_celltype", "post_celltype", "pre_celltype_merged", "post_celltype_merged",  \
                                             "pre_id_mapped", "post_id_mapped", "post_compartment", "overlap_volume"])

df_synapses_indexed.sort_index(inplace=True)
df_synapses_indexed.synapse_count.sum()

Load or initialize local overlap volumes and global index data

In [ ]:
index_data = compile_index_data(eval_folder, df_synapses_indexed)

### Define models

In [ ]:
model_P_impl = Model(index_data, ["pre_celltype_merged", "post_celltype_merged"])
model_S_impl = Model(index_data, ["post_compartment"])
model_C_impl = Model(index_data, ["pre_id_mapped", "post_id_mapped"])
model_Ca_impl = Model(index_data, ["pre_id_mapped"])
model_Cb_impl = Model(index_data, ["post_id_mapped"])

### Run models

In [ ]:
statistics = H01AggregateStatistics(index_data)
multilevel_analysis = MultilevelAnalysis(index_data, df_synapses_indexed, statistics, num_realizations=10000)
multilevel_analysis.run_null_and_empirical()

In [ ]:
multilevel_analysis.run_model(
    MODEL_NULL,
    model_P_impl,
    MODEL_P
)

In [ ]:
multilevel_analysis.run_model(
    MODEL_P,
    model_S_impl,
    MODEL_PS
)

In [ ]:
multilevel_analysis.run_model(
    MODEL_PS,
    model_C_impl,
    MODEL_PSC
)

In [ ]:
multilevel_analysis.run_model(
    MODEL_PS,
    model_Ca_impl,
    MODEL_PSCa
)

In [ ]:
multilevel_analysis.run_model(
    MODEL_PS,
    model_Cb_impl,
    MODEL_PSCb
)

#### Save results

In [ ]:
filename = eval_folder / "multilevel_analysis.pkl"
with open(filename, 'wb') as file:
    pickle.dump(multilevel_analysis, file)